In [1]:
print("Hallo, es klappt!")

Hallo, es klappt!


In [3]:
!pip install requests beautifulsoup4 pandas

     -------------------------------------- 64.7/64.7 kB 698.7 kB/s eta 0:00:00
     -------------------------------------- 107.7/107.7 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 11.3/11.3 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 107.0/107.0 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 71.0/71.0 kB 1.9 MB/s eta 0:00:00
     -------------------------------------- 131.1/131.1 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 159.4/159.4 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 13.1/13.1 MB 3.1 MB/s eta 0:00:00
     -------------------------------------- 509.2/509.2 kB 3.5 MB/s eta 0:00:00
     -------------------------------------- 347.8/347.8 kB 1.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = "https://www.scrapethissite.com/pages/forms/"


In [5]:
params = {"page_num": 1}

response = requests.get(BASE_URL, params=params)
response.raise_for_status()

print("Status-Code:", response.status_code)
print(response.text[:500])  # zeigt die ersten 500 Zeichen


Status-Code: 200
<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping</title>
    <link rel="icon" type="image/png" href="/static/images/scraper-icon.png" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="description" content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common webs


In [6]:
soup = BeautifulSoup(response.text, "html.parser")

# 1) Tabelle finden
table = soup.find("table")  # es gibt nur eine relevante Tabelle

# 2) Spaltenüberschriften auslesen
header_cells = table.find_all("th")
columns = [th.get_text(strip=True) for th in header_cells]
print("Spaltennamen:", columns)

# 3) Datenzeilen auslesen (jede Zeile hat Klasse 'team')
rows = table.select("tr.team")
print("Anzahl Zeilen auf Seite 1:", len(rows))

# 4) Beispiel: erste Zeile anzeigen
first_row = [td.get_text(strip=True) for td in rows[0].find_all("td")]
print("Erste Datenzeile:", first_row)


Spaltennamen: ['Team Name', 'Year', 'Wins', 'Losses', 'OT Losses', 'Win %', 'Goals For (GF)', 'Goals Against (GA)', '+ / -']
Anzahl Zeilen auf Seite 1: 25
Erste Datenzeile: ['Boston Bruins', '1990', '44', '24', '', '0.55', '299', '264', '35']


In [7]:
all_rows = []
page = 1

while True:
    params = {"page_num": page}
    resp = requests.get(BASE_URL, params=params)
    resp.raise_for_status()
    
    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table")
    
    # sicherheitshalber: wenn keine Tabelle da ist -> abbrechen
    if table is None:
        print(f"Keine Tabelle auf Seite {page} gefunden – Abbruch.")
        break
    
    rows = table.select("tr.team")
    
    # wenn keine Datenzeilen mehr -> fertig
    if not rows:
        print(f"Keine Zeilen mehr auf Seite {page} – Abbruch.")
        break
    
    for row in rows:
        cells = [td.get_text(strip=True) for td in row.find_all("td")]
        all_rows.append(cells)
    
    print(f"Seite {page}: {len(rows)} Zeilen gesammelt (gesamt: {len(all_rows)})")
    
    page += 1

print("Fertig. Insgesamt Zeilen:", len(all_rows))


Seite 1: 25 Zeilen gesammelt (gesamt: 25)
Seite 2: 25 Zeilen gesammelt (gesamt: 50)
Seite 3: 25 Zeilen gesammelt (gesamt: 75)
Seite 4: 25 Zeilen gesammelt (gesamt: 100)
Seite 5: 25 Zeilen gesammelt (gesamt: 125)
Seite 6: 25 Zeilen gesammelt (gesamt: 150)
Seite 7: 25 Zeilen gesammelt (gesamt: 175)
Seite 8: 25 Zeilen gesammelt (gesamt: 200)
Seite 9: 25 Zeilen gesammelt (gesamt: 225)
Seite 10: 25 Zeilen gesammelt (gesamt: 250)
Seite 11: 25 Zeilen gesammelt (gesamt: 275)
Seite 12: 25 Zeilen gesammelt (gesamt: 300)
Seite 13: 25 Zeilen gesammelt (gesamt: 325)
Seite 14: 25 Zeilen gesammelt (gesamt: 350)
Seite 15: 25 Zeilen gesammelt (gesamt: 375)
Seite 16: 25 Zeilen gesammelt (gesamt: 400)
Seite 17: 25 Zeilen gesammelt (gesamt: 425)
Seite 18: 25 Zeilen gesammelt (gesamt: 450)
Seite 19: 25 Zeilen gesammelt (gesamt: 475)
Seite 20: 25 Zeilen gesammelt (gesamt: 500)
Seite 21: 25 Zeilen gesammelt (gesamt: 525)
Seite 22: 25 Zeilen gesammelt (gesamt: 550)
Seite 23: 25 Zeilen gesammelt (gesamt: 575)


In [8]:
df = pd.DataFrame(all_rows, columns=columns)

# numerische Spalten in Zahlen umwandeln
numeric_cols = [
    "Year",
    "Wins",
    "Losses",
    "OT Losses",
    "Win %",
    "Goals For (GF)",
    "Goals Against (GA)",
    "+ / -"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df.head()


,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,NaN,0.550,299,264,35
1,Buffalo Sabres,1990,31,30,NaN,0.388,292,278,14
2,Calgary Flames,1990,46,26,NaN,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,NaN,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,NaN,0.425,273,298,-25


In [9]:
df.to_csv("data.csv", index=False)
print("CSV gespeichert als data.csv")


CSV gespeichert als data.csv


In [10]:
df = pd.read_csv("data.csv")
df.head()


,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),+ / -
0,Boston Bruins,1990,44,24,NaN,0.550,299,264,35
1,Buffalo Sabres,1990,31,30,NaN,0.388,292,278,14
2,Calgary Flames,1990,46,26,NaN,0.575,344,263,81
3,Chicago Blackhawks,1990,49,23,NaN,0.613,284,211,73
4,Detroit Red Wings,1990,34,38,NaN,0.425,273,298,-25


In [12]:
# How made the most "wins" in 1990, 2000, and 2010?
years = [1990, 2000, 2010]

for year in years:
    subset = df[df["Year"] == year]
    max_wins = subset["Wins"].max()
    top_teams = subset[subset["Wins"] == max_wins]["Team Name"].tolist()
    
    print(f"Jahr {year}:")
    print(f"  Maximale Anzahl Wins: {max_wins}")
    print(f"  Team(s) mit den meisten Wins: {', '.join(top_teams)}")
    print()


Jahr 1990:
  Maximale Anzahl Wins: 49
  Team(s) mit den meisten Wins: Chicago Blackhawks

Jahr 2000:
  Maximale Anzahl Wins: 52
  Team(s) mit den meisten Wins: Colorado Avalanche

Jahr 2010:
  Maximale Anzahl Wins: 54
  Team(s) mit den meisten Wins: Vancouver Canucks



In [13]:
# How many teams participated in 1991, 2001, and 2011?
years_participation = [1991, 2001, 2011]

for year in years_participation:
    subset = df[df["Year"] == year]          # Filter auf das Jahr
    n_teams = subset["Team Name"].nunique()  # Anzahl einzigartiger Teams
    print(f"Jahr {year}: {n_teams} Teams haben teilgenommen.")


Jahr 1991: 22 Teams haben teilgenommen.
Jahr 2001: 30 Teams haben teilgenommen.
Jahr 2011: 30 Teams haben teilgenommen.
